# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import _data_utils
import _script_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))

In [6]:

# df = utils.read_data_all()
proj = _data_utils.add_new_codes(proj)
    
    ## function that adds known agency name to df 
df = _script_utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [7]:
df.sample(3)

,unnamed:_0,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
1134,NaN,2022-03-30,Z40E,NaN,5157121,0621000122L,INTERSECTION OF HOWARD AVE AND PINE STREET AND INTERSECTION OF Q STREET AND YOSEMITE AVE. TRAFFIC SIGNAL MODIFICATIONS AND INTERSECTIONS IMPROVEMENTS,39,Cong Dist 16,Active,"INTERSECTION OF HOWARD AVE AND PINE STREET AND INTERSECTION OF Q STREET AND YOSEMITE AVE. TRAFFIC SIGNAL MODIFICATIONS AND INTERSECTIONS IMPROVEMENTS AT HOWARD ROAD, OLIVE AVENUE, PINE STREET, YOSEMITE AVENUE & Q STREET. PURCHASE AND INSTALL ADAPTIVE SIGNAL CONTROL TECHNOLOGY.",15,Preliminary Engineering,15000.00,13000.00,L5157MCTC,5157.0,Madera,6.0,Madera County,Madera County Transportation Commission - RTPA,Madera County Transportation Commission
8108,NaN,2022-11-23,ZS30,NaN,X081029,0415000086S,"IN SAN MATEO COUNTY, ON ROUTES 82 AND 84 AT VARIOUS LOCATIONS. INSTALL CROSSWALK SAFETY ENHANCEMENTS (TC)",81,Cong Dist 14,Closed,"ON STATE ROUTE: 82, 84. IN SAN MATEO COUNTY, ON ROUTES 82 AND 84 AT VARIOUS LOCATIONS. INSTALL CROSSWALK SAFETY ENHANCEMENTS (TC)",21,Safety,2120012.73,2048122.83,S MTC,NaN,California,4.0,San Mateo County,NaN,NaN
9325,NaN,2023-01-23,Z001,NaN,P057064,0714000067S,"IN AND NEAR DIAMOND BAR, FROM THE ORANGE COUNTY LINE TO RTE 60. PAVEMENT REHABILITATION (G13 CONTINGENCY PROJECT) PAVEMENT REHABILITATION (TC)",59,Cong Dist 39,Active,"IN AND NEAR DIAMOND BAR, FROM THE ORANGE COUNTY LINE TO RTE 60. PAVEMENT REHABILITATION (G13 CONTINGENCY PROJECT) PAVEMENT REHABILITATION (TC)",15,Preliminary Engineering,15218.46,15218.46,S SCAG,NaN,California,12.0,Orange County,NaN,NaN


## Test & Export

In [8]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

In [9]:
agg.sample(10)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
376,2022-02-10,P222001,California,S FSTIP,L030,nan,01380504S,11,Bridge Replacement - No Added Capacity,IN MEN CO NEAR UKIAH @ RUSSIAN RVR BR 10-80 REPLACE BRIDGE (TC),172468.08,Cong Dist 2,1.0,45,Mendocino County,Mendocino County,nan,None,None,Replace Bridge in Mendocino County,"This project is part of the nan Program, and recieved $ 172468.08. This project will Replace Bridge in Mendocino County"
242,2022-02-08,0101827,California,S SCAG,M001,nan,0713000007S,16,Right of Way,RTE 10 IN LA COUNTY FROM CITRUS ST IN W COVINA TO RTE 57 IN POMONA HOV LANES (TC),1025000.00,Cong Dist 32|Cong Dist 35,7.0,37,Los Angeles County,Los Angeles County,nan,None,None,HOV Lane in Los Angeles County,"This project is part of the nan Program, and recieved $ 1025000.0. This project will HOV Lane in Los Angeles County"
1712,2022-05-25,X077027,California,S SJCOG,Z001,nan,1013000247S,6|17|42,4R - Restoration & Rehabilitation|Construction Engineering|Training,"SAN JOAQUIN COUNTY AT VARIOUS LOCATIONS MVP'S, GORE PAVING AND SLOPE PAVING",2904875.93,Cong Dist 0,nan,999,Statewide,Statewide,nan,None,None,Road Restoration & Rehabilitation Statewide,"This project is part of the nan Program, and recieved $ 2904875.93. This project will Road Restoration & Rehabilitation Statewide"
5878,2023-03-06,6207074,Caltrans,S6207SCAG,Q920,nan,0714000278L,1|15|17,New Construction Roadway|Preliminary Engineering|Construction Engineering,"ROUTE 002: FROM 0.5 MILES SOUTH OF BRANDEN STREET TO I-5/SR2 INTERCHANGE MODIFY TERMINUS, SOUTH WALLS, LANDSCAPING, INSTALLING DETECTOR LOOPS AND RAM",2126000.00,Cong Dist 28,7.0,37,Multi-County,Multi-County,6207.0,CT-ADMIN,CT-ADMIN,Caltrans New Construction Roadway,"This project is part of the nan Program, and recieved $ 2126000.0. This project will Caltrans New Construction Roadway"
1012,2022-04-04,9103001,California,S AMBAG,3410,nan,05027102/9S1,16,Right of Way,"HOLLISTER, SR-156, UNION MITCHELL RD/SA BRA R/W; CONST 2-LANE EXPWY/NEW AL",960000.00,Cong Dist 20,5.0,69,San Benito County,San Benito County,nan,None,None,Right of Way Project in San Benito County,"This project is part of the nan Program, and recieved $ 960000.0. This project will Right of Way Project in San Benito County"
4223,2022-11-10,5449026,Highland,L5449SCAG,LU2E,nan,0812000181L,15,Preliminary Engineering,"MCKINLEY STREET FROM 9TH STREET TO BASELINE AND AT THE INTERSECTION OF BASE LINE AND VALARIA DRIVE CONSTRUCT SIDEWALKS, CURB AND GUTTERS AND CURB RAM",21693.62,Cong Dist 8,8.0,71,San Bernardino County,San Bernardino County,5449.0,San Bernardino Associated Governments,Southern California Association Of Governments,Construct Sidewalk in Highland,"This project is part of the nan Program, and recieved $ 21693.62. This project will Construct Sidewalk in Highland"
5634,2023-02-22,5168015,Calexico,L5168SCAG,HY10,nan,11956606L,15,Preliminary Engineering,3 MILE REACH OF NEW RIVER FROM INTERNATIONAL BORDER TO SR-98 IN CALEXICO BIKE PATH AND PUBLIC PARK SPACE,281805.56,Cong Dist 51,11.0,25,Imperial County,Imperial County,5168.0,Imperial County Transportation Commission,Southern California Association Of Governments,New Bike Lanes in Calexico,"This project is part of the nan Program, and recieved $ 281805.56. This project will New Bike Lanes in Calexico"
2237,2022-07-01,0010180,California,S SHASTA,22SP,Statewide and Nonmetropolitan Transportation Planning (FY 22),0021000344S,18,Planning,OWP 21/22- SHASTA REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR SHASTA REGIONAL TRANSPORTATION AGENCY (SRTA) OVERALL WORK PROGRAM (OWP) FOR FI,291819.00,Cong Dist 0,nan,999,Statewide,Statewide,nan,N

In [10]:
sorted(list(agg.project_title_new))

[' Added Roadway Capacity in Industry',
 ' Added Roadway Capacity in Kern County',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Madera County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Murrieta',
 ' Added Roadway Capacity in Murrieta',
 ' Added Roadway 

In [11]:
len(agg)

5945

In [20]:
agg = _script_utils.title_column_names(agg)

In [21]:
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_agg.csv")

In [14]:
# ###test full. PASS title, no concat and 1241 rows
# full = _script_utils.get_clean_data(df, full_or_agg = 'full')

In [16]:
full>>filter(_["Recipient Project Number"]=="0118000055L")

,Unnamed: 0,Fmis Transaction Date,Program Code,Program Code Description,Project Number,Recipient Project Number,Project Title,County Code,Congressional District,Project Status Description,Project Description,Improvement Type,Improvement Type Description,Total Cost Amount,Obligations Amount,Summary Recipient Defined Text Field 1 Value,Implementing Agency Locode,Implementing Agency,District,County Name,Rtpa Name,Mpo Name,County Name Title,Project Title New
5626,NaN,2023-01-10,ER01,nan,32L0320,0118000055L,"IN HUMBOLDT COUNTY, MATTOLE RD PM 9.00 PERMANENT RESTORATION",23,Cong Dist 2,Closed,"IN HUMBOLDT COUNTY, MATTOLE RD PM 9.00 PERMANENT RESTORATION",4,4R - No Added Capacity,24675.98,21845.64,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County,Road Restoration & Rehabilitation in Humboldt County
5632,NaN,2023-01-10,ER14,nan,32L0320,0118000055L,"IN HUMBOLDT COUNTY, MATTOLE RD PM 9.00 PERMANENT RESTORATION",23,Cong Dist 2,Closed,"IN HUMBOLDT COUNTY, MATTOLE RD PM 9.00 PERMANENT RESTORATION",4,4R - No Added Capacity,88780.00,78596.93,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County,Road Restoration & Rehabilitation in Humboldt County
5633,NaN,2023-01-10,ER14,nan,32L0320,0118000055L,"IN HUMBOLDT COUNTY, MATTOLE RD PM 9.00 PERMANENT RESTORATION",23,Cong Dist 2,Closed,"IN HUMBOLDT COUNTY, MATTOLE RD PM 9.00 PERMANENT RESTORATION",17,Construction Engineering,8583.23,7598.73,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County,Road Restoration & Rehabilitation in Humboldt County
5644,NaN,2023-01-10,Z9W0,nan,32L0320,0118000055L,"IN HUMBOLDT COUNTY, MATTOLE RD PM 9.00 PERMANENT RESTORATION",23,Cong Dist 2,Closed,"IN HUMBOLDT COUNTY, MATTOLE RD PM 9.00 PERMANENT RESTORATION",15,Preliminary Engineering,7234.49,5938.41,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County,Road Restoration & Rehabilitation in Humboldt County


In [11]:
full = _script_utils.title_column_names(full)

In [24]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_all.csv")